#### RiboScore calculate

In [ ]:
import pandas as pd

In [ ]:
# human
hp1_1 = pd.read_csv('./Data/hp1-1.Read1.bin.gz',sep='\t')
hp1_1_ = hp1_1.iloc[:,1:]
hp1_1_colsum = hp1_1_.sum(axis=0).reset_index()
hp1_1_colsum.columns = ['category','umi']

hp1_2 = pd.read_csv('./Data/hp1-2.Read1.bin.gz',sep='\t')
hp1_2_ = hp1_2.iloc[:,1:]
hp1_2_colsum = hp1_2_.sum(axis=0).reset_index()
hp1_2_colsum.columns = ['category','umi']

hp2_1 = pd.read_csv('./Data/hp2-1.Read1.bin.gz',sep='\t')
hp2_1_ = hp2_1.iloc[:,1:]
hp2_1_colsum = hp2_1_.sum(axis=0).reset_index()
hp2_1_colsum.columns = ['category','umi']

hp2_2 = pd.read_csv('./Data/hp2-2.Read1.bin.gz',sep='\t')
hp2_2_ = hp2_2.iloc[:,1:]
hp2_2_colsum = hp2_2_.sum(axis=0).reset_index()
hp2_2_colsum.columns = ['category','umi']

In [ ]:
# filter low expressed sequence
df = hp1_1.copy()
tmp_df = df.iloc[:,1:]
tmp_df[tmp_df < 3] = np.nan
tmp_df.insert(0, 'variable',df['variable'])
hp1_1 = tmp_df

df = hp1_2.copy()
tmp_df = df.iloc[:,1:]
tmp_df[tmp_df < 3] = np.nan
tmp_df.insert(0, 'variable',df['variable'])
hp1_2 = tmp_df

df = hp2_1.copy()
tmp_df = df.iloc[:,1:]
tmp_df[tmp_df < 6] = np.nan
tmp_df.insert(0, 'variable',df['variable'])
hp2_1 = tmp_df

df = hp2_2.copy()
tmp_df = df.iloc[:,1:]
tmp_df[tmp_df < 6] = np.nan
tmp_df.insert(0, 'variable',df['variable'])
hp2_2 = tmp_df

In [ ]:
df_1 = hp1_1.dropna(thresh=10)
df_1.columns = ['sequence'] + list(map(lambda x: 'Rep1Bin' + str(x), range(1,11)))
df_2 = hp1_2.dropna(thresh=10)
df_2.columns = ['sequence'] + list(map(lambda x: 'Rep2Bin' + str(x), range(1,11)))
merge_df = pd.merge(df_1,df_2,on='sequence')
print("hp1 sequence count: " + str(merge_df.shape[0]))
merge_df.index = merge_df['sequence']
del merge_df['sequence']
merge_df = merge_df.fillna(0)
merge_df.to_csv("./Polysomes_hp1_RawSeq_df.gz",compression="gzip",sep="\t")

df_1 = hp5_1.dropna(thresh=10)
df_1.columns = ['sequence'] + list(map(lambda x: 'Rep1Bin' + str(x), range(1,11)))
df_2 = hp5_2.dropna(thresh=10)
df_2.columns = ['sequence'] + list(map(lambda x: 'Rep2Bin' + str(x), range(1,11)))
merge_df = pd.merge(df_1,df_2,on='sequence')
print("hp2 sequence count: " + str(merge_df.shape[0]))
merge_df.index = merge_df['sequence']
del merge_df['sequence']
merge_df = merge_df.fillna(0)
merge_df.to_csv("./Data/Polysomes_hp2_RawSeq_df.gz",compression="gzip",sep="\t")

##### Normalization

In [ ]:
Normalized_hp1_df = pd.read_csv("./Result/Polysomes_hp1_Normalized_df.txt",sep="\t")
Normalized_hp2_df = pd.read_csv("./Result/Polysomes_hp2_Normalized_df.txt",sep="\t")

In [ ]:
# drop Bin1 of region 2
del Normalized_hp2_df['Rep1Bin1']
del Normalized_hp2_df['Rep2Bin1']

Normalized_hp1_df.index = Normalized_hp1_df['Unnamed: 0']
del Normalized_hp1_df['Unnamed: 0']
Normalized_hp2_df.index = Normalized_hp2_df['Unnamed: 0']
del Normalized_hp2_df['Unnamed: 0']

In [ ]:
# human region 1
Polysomes_DESeq2Normalized_df = Normalized_hp1_df.copy()
Ribosome_df = pd.DataFrame([[0,0,0,0,0,1,1,2,3,4]]*Polysomes_DESeq2Normalized_df.shape[0],
                           columns = list(map(lambda x: 'Bin' + str(x), range(1,11))),index=Polysomes_DESeq2Normalized_df.index)
Poly_Norm_rep1 = Polysomes_DESeq2Normalized_df.iloc[:,0:10]
Poly_Norm_rep2 = Polysomes_DESeq2Normalized_df.iloc[:,10:20]
Poly_Norm_rep1.columns = list(map(lambda x: 'Bin' + str(x), range(1,11)))
Poly_Norm_rep2.columns = list(map(lambda x: 'Bin' + str(x), range(1,11)))

Poly_Norm_df_rowsum = Poly_Norm_rep1.sum(axis=1)
Poly_Norm_Fraction = Poly_Norm_rep1.div(Poly_Norm_df_rowsum,axis=0)
Poly_Norm_df_Ribosome = Poly_Norm_Fraction.mul(Ribosome_df)
Poly_Norm_df_Ribosome_rowsum = Poly_Norm_df_Ribosome.sum(axis=1)
Poly_Ribosome_Score_df = Poly_Norm_df_Ribosome_rowsum.reset_index()
Poly_Ribosome_Score_df.columns = ['sequence','RiboScore']
Poly_Ribosome_Score_df.to_csv("./Result/Ploysomes_hp1_1_RiboScore.csv",sep="\t",index=False)

Poly_Norm_df_rowsum = Poly_Norm_rep2.sum(axis=1)
Poly_Norm_Fraction = Poly_Norm_rep2.div(Poly_Norm_df_rowsum,axis=0)
Poly_Norm_df_Ribosome = Poly_Norm_Fraction.mul(Ribosome_df)
Poly_Norm_df_Ribosome_rowsum = Poly_Norm_df_Ribosome.sum(axis=1)
Poly_Ribosome_Score_df = Poly_Norm_df_Ribosome_rowsum.reset_index()
Poly_Ribosome_Score_df.columns = ['sequence','RiboScore']
Poly_Ribosome_Score_df.to_csv("./Result/Ploysomes_hp1_2_RiboScore.csv",sep="\t",index=False)

# human region 2
Polysomes_DESeq2Normalized_df = Normalized_hp2_df.copy()
Ribosome_df = pd.DataFrame([[0,0,0,0,1,1,2,3,4]]*Polysomes_DESeq2Normalized_df.shape[0],
                           columns = list(map(lambda x: 'Bin' + str(x), range(2,11))),index=Polysomes_DESeq2Normalized_df.index)
Poly_Norm_rep1 = Polysomes_DESeq2Normalized_df.iloc[:,0:9]
Poly_Norm_rep2 = Polysomes_DESeq2Normalized_df.iloc[:,9:18]
Poly_Norm_rep1.columns = list(map(lambda x: 'Bin' + str(x), range(2,11)))
Poly_Norm_rep2.columns = list(map(lambda x: 'Bin' + str(x), range(2,11)))

Poly_Norm_df_rowsum = Poly_Norm_rep1.sum(axis=1)
Poly_Norm_Fraction = Poly_Norm_rep1.div(Poly_Norm_df_rowsum,axis=0)
Poly_Norm_df_Ribosome = Poly_Norm_Fraction.mul(Ribosome_df)
Poly_Norm_df_Ribosome_rowsum = Poly_Norm_df_Ribosome.sum(axis=1)
Poly_Ribosome_Score_df = Poly_Norm_df_Ribosome_rowsum.reset_index()
Poly_Ribosome_Score_df.columns = ['sequence','RiboScore']
Poly_Ribosome_Score_df.to_csv("./Result/Ploysomes_hp2_1_RiboScore.csv",sep="\t",index=False)

Poly_Norm_df_rowsum = Poly_Norm_rep2.sum(axis=1)
Poly_Norm_Fraction = Poly_Norm_rep2.div(Poly_Norm_df_rowsum,axis=0)
Poly_Norm_df_Ribosome = Poly_Norm_Fraction.mul(Ribosome_df)
Poly_Norm_df_Ribosome_rowsum = Poly_Norm_df_Ribosome.sum(axis=1)
Poly_Ribosome_Score_df = Poly_Norm_df_Ribosome_rowsum.reset_index()
Poly_Ribosome_Score_df.columns = ['sequence','RiboScore']
Poly_Ribosome_Score_df.to_csv("./Result/Ploysomes_hp2_2_RiboScore.csv",sep="\t",index=False)